<a href="https://colab.research.google.com/github/BahodirML/Coding_Practices/blob/main/VertexYolov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth

BUCKET_NAME = "my-vertex-ai-project-poised-conduit-420907-unique-20240421"
DATA_FILE = "my-vertex-ai-project-poised-conduit-420907-unique-20240421/face_mask.v1i.yolov8"

def download_files(DATA_DIR):
  auth.authenticate_user()
  !gsutil -m cp -r gs://{DATA_DIR} /content/
  return 'done'

download_files(DATA_FILE)

In [ ]:
import os
import cv2
import glob
import shutil

def preprocess_data(image_dir, label_dir, data_yaml, is_train=True, copy_yaml=True):
    dataset_type = 'train' if is_train else 'valid'
    output_image_dir = os.path.join('data', dataset_type, 'images')
    output_label_dir = os.path.join('data', dataset_type, 'labels')

    jpeg_images = glob.glob(os.path.join(image_dir, '*.jpg'))
    for image_path in jpeg_images:
        image = cv2.imread(image_path)
        image = cv2.resize(image, (416, 416))
        relative_path = os.path.relpath(image_path, image_dir)
        output_image_folder = os.path.join(output_image_dir, os.path.dirname(relative_path))
        os.makedirs(output_image_folder, exist_ok=True)
        output_image_path = os.path.join(output_image_folder, os.path.basename(image_path))
        cv2.imwrite(output_image_path, image)
        print(f"Processed and saved: {output_image_path}")

    output_label_dir = os.path.join('data', dataset_type, 'labels')
    shutil.copytree(label_dir, output_label_dir)
    print(f"Copied labels to: {output_label_dir}")

    if copy_yaml:
        output_yaml_dir = 'data'
        shutil.copy(data_yaml, output_yaml_dir)
        print(f"Copied yaml to: {output_yaml_dir}")




image_dir_train = '/content/face_mask.v1i.yolov8/train/images'
label_dir_train = '/content/face_mask.v1i.yolov8/train/labels'
yaml = '/content/face_mask.v1i.yolov8/data.yaml'
preprocess_data(image_dir_train, label_dir_train, yaml, copy_yaml=True, is_train=True)

image_dir_test = '/content/face_mask.v1i.yolov8/test/images'
label_dir_test = '/content/face_mask.v1i.yolov8/test/labels'
yaml = '/content/face_mask.v1i.yolov8/data.yaml'
preprocess_data(image_dir_test, label_dir_test, yaml, is_train=False)

In [6]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.0/755.0 kB 18.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-man

In [ ]:
from ultralytics import YOLO
import os
from IPython.display import display, Image
from IPython import display
display.clear_output()
!yolo mode = cheks

def train_model():
  !yolo task = detect mode = train model = yolov8m.pt data=/content/data/data.yaml epochs=10 imgsz=416

In [8]:

!yolo task = detect mode = predict model = /content/runs/detect/train/weights/best.pt conf = 0.5 source=/content/data/data.yaml

Ultralytics YOLOv8.2.5 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs

Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/cfg/__init__.py", line 582, in entrypoint
    getattr(model, mode)(**overrides)  # default args from model
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/engine/model.py", line 452, in predict
    return self.predictor.predict_cli(source=source) if is_cli else self.predictor(source=source, stream=stream)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/engine/predictor.py", line 177, in predict_cli
    for _ in gen:  # noqa, running CLI inference without accumulating any outputs (do not modify)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 35, in generator_context
    response = gen.send(None)

In [ ]:

import glob
from IPython.display import display, Image

for image_path in glob.glob(f'/content/runs/detect/predict/*.jpg'):
  display(Image(filename=image_path, height=600))
  print("\n")
